In [15]:
import sympy as sp
import numpy as np
from scipy import integrate, optimize
import matplotlib.pyplot as plt

#  1) Формулы прямоугольников

In [276]:
x = sp.symbols('x')
n = 16
f = x*x*x*sp.exp(-0.01*n*x**(3/2))
a = 0.1*(n+3)
b = 0.25*(n+3)
eps = 0.001

In [277]:
df = sp.diff(f, x)
d2f = sp.diff(df, x)

In [278]:
f1 = sp.lambdify(x, -df) # Минус т.к. мы ищем максимум функции
f2 = sp.lambdify(x, -d2f)

M1 = abs(optimize.minimize_scalar(f1, bounds = (a, b),method='Bounded').fun)
M2 = abs(optimize.minimize_scalar(f2, bounds = (a, b),method='Bounded').fun)
N1 = int(((b-a)**(2)*M1)/(2*eps))
N2 = int(((b-a)**(2)*M2)/(24*eps))

In [279]:
x_num = np.linspace(a, b, N1)
x_num_for_middle = np.linspace(a, b, N2)

In [280]:
sum_left = sum([(x_num[i+1] - x_num[i]) * f.subs({x:x_num[i]}) for i in range(len(x_num) - 1)])
sum_right = sum([(x_num[i+1] - x_num[i]) * f.subs({x:x_num[i+1]}) for i in range(len(x_num) - 1)])
sum_middle = sum([(x_num_for_middle[i+1] - x_num_for_middle[i]) * f.subs({x:(x_num_for_middle[i]+x_num_for_middle[i+1])/2}) 
                  for i in range(len(x_num_for_middle) - 1)])

In [281]:
sum_left, sum_right, sum_middle

(38.2351201672786, 38.2367300788330, 38.2359263403804)

In [282]:
f_lam = sp.lambdify(x, f)
v, err = integrate.quad(f_lam, a, b)
print(v)

38.23592512595571


In [283]:
print(abs(sum_left - v) < eps)
print(abs(sum_right - v) < eps)
print(abs(sum_middle - v) < eps)

True
True
True


# 2) Формула трапеций

In [339]:
x = sp.symbols('x')
n = 16
f = (x - 4.75)*(x - 3.8)*(x - 2.85)*(x - 1.9)/(9.774075 - 5.14425*x)
a = 0.1*(n+3)
b = 0.25*(n+3)
eps = 0.001

In [340]:
df = sp.diff(f, x)
d2f = sp.diff(df, x)

In [341]:
f1 = sp.lambdify(x, -d2f) # Минус т.к. мы ищем максимум функции

M = abs(optimize.minimize_scalar(f1, bounds = (a, b), method='Bounded').fun)
N = int((((b-a)**(3)*M)/(12*eps))**(1/2))
N

134

In [342]:
x_num = np.linspace(a, b, N)

In [343]:
result = sum([(x_num[i+1] - x_num[i]) * (f.subs({x:x_num[i]}) + f.subs({x:x_num[i+1]})) / 2 for i in range(len(x_num) - 1)])

In [344]:
result

0.352485753725832

In [345]:
f_lam = sp.lambdify(x, f)
v, err = integrate.quad(f_lam, a, b)
print(v)

0.3562499999999993


In [346]:
abs(result - v) < eps

False

# 3) Формула Симпсона

In [255]:
x = sp.symbols('x')
n = 16
f = x*x*x*sp.exp(-0.01*n*x**(3/2))
a = 0.1*(n+3)
b = 0.25*(n+3)
eps = 0.001

In [256]:
df = sp.diff(f, x)
d2f = sp.diff(df, x)
d3f = sp.diff(d2f, x)
d4f = sp.diff(d3f, x)

In [257]:
f1 = sp.lambdify(x, -d4f) # Минус т.к. мы ищем максимум функции

M = abs(optimize.minimize_scalar(f1, bounds = (a, b),method='Bounded').fun) 
N = int((((b-a)**(5)*M)/(2880*eps))**(1/4))
N

3

In [258]:
x_num = np.linspace(a, b, N + 1)
h = (b-a)/N

In [259]:
result = h/6*(f.subs({x:x_num[0]}) + f.subs({x:x_num[len(x_num) - 1]}) + 2*(sum([f.subs({x:x_num[i]}) for i in range(1, len(x_num) - 1)]))
              + 4*(sum([f.subs({x: (x_num[i]  + x_num[i + 1]) / 2}) for i in range(len(x_num) - 1)])))

In [260]:
result

38.2367342335588

In [261]:
f_lam = sp.lambdify(x, f)
v, err = integrate.quad(f_lam, a, b)
print(v)

38.23592512595571


In [262]:
abs(result - v) < eps

True

# 4) Формулы интерполяционного типа

In [97]:
x = sp.symbols('x')
n = 16
f = x*x*x*sp.exp(-0.01*n*x**(3/2))
a = 0.1*(n+3)
b = 0.25*(n+3)
eps = 0.001
N = 4

In [62]:
df = sp.diff(f, x)

In [135]:
x_num = np.linspace(a, b, N)

In [110]:
w = 1
for i in range(N):
    w *= (x - x_num[i])
w

(x - 4.75)*(x - 3.8)*(x - 2.85)*(x - 1.9)

In [65]:
dw = sp.diff(w, x)
C_list = [(w / (x - x_num[i]) / dw.subs({x:x_num[i]})) for i in range(N)]
C_list

[-0.194391796666181*(x - 4.75)*(x - 3.8)*(x - 2.85),
 0.583175389998542*(x - 4.75)*(x - 3.8)*(x - 1.9),
 -0.583175389998542*(x - 4.75)*(x - 2.85)*(x - 1.9),
 0.194391796666181*(x - 3.8)*(x - 2.85)*(x - 1.9)]

In [121]:
def trapezoid(f, a, b):
    df = sp.diff(f, x)
    d2f = sp.diff(df, x)
    
    f1 = sp.lambdify(x, -d2f) # Минус т.к. мы ищем максимум функции

    M = abs(optimize.minimize_scalar(f1, bounds = (a, b),method='Bounded').fun)
    N = int((((b-a)**(3)*M)/(12*eps))**(1/2))
    x_num = np.linspace(a, b, N)
    
    return sum([(x_num[i+1] - x_num[i]) * (f.subs({x:x_num[i]}) + f.subs({x:x_num[i+1]})) / 2 for i in range(len(x_num) - 1)])

def abs_trapezoid(f, a, b):
    df = sp.diff(f, x)
    d2f = sp.diff(df, x)
    
    f1 = sp.lambdify(x, -d2f) # Минус т.к. мы ищем максимум функции

    M = abs(optimize.minimize_scalar(f1, bounds = (a, b),method='Bounded').fun)
    N = int((((b-a)**(3)*M)/(12*eps))**(1/2))
    x_num = np.linspace(a, b, N)
    
    return sum([abs((x_num[i+1] - x_num[i]) * (f.subs({x:x_num[i]}) + f.subs({x:x_num[i+1]})) / 2) for i in range(len(x_num) - 1)])

In [115]:
C_list_integrate = [trapezoid(C_list[i], a, b) for i in range(N)]

In [68]:
C_list_integrate

[0.356510925292969, 1.06862093950006, 1.06862093950006, 0.356510925292969]

In [69]:
result = sum([C_list_integrate[i] * f.subs({x:x_num[i]}) for i in range(N)])

In [70]:
result

38.2797220221449

In [71]:
f_lam = sp.lambdify(x, f)
v, err = integrate.quad(f_lam, a, b)
print(v)

38.23592512595571


In [54]:
abs(result - v) < eps

False

In [117]:
dff = sp.diff(f)

for i in range(N):
    dff = sp.diff(dff)

f5 = sp.lambdify(x, -dff)

M_new = abs(optimize.minimize_scalar(f5, bounds = (a, b),method='Bounded').fun)
tmp = M_new/np.prod([i for i in range(1, N+2)])
tmp 

0.03384252683144602

In [122]:
tmp * abs_trapezoid(w)

0.0427584202298634

In [123]:
result - v

0.0437968961892210

### Со встроенным интегралом

In [55]:
C_list_integrate_2 = [sp.lambdify(x, i) for i in C_list]
C_list_integrate_2 = [integrate.quad(i, a, b)[0] for i in C_list_integrate_2]

In [56]:
C_list_integrate_2

[0.2216666666666659,
 1.0133333333333332,
 0.3799999999999994,
 1.0133333333333339,
 0.22166666666666687]

In [57]:
result_2 = sum([C_list_integrate_2[i] * f.subs({x:x_num[i]}) for i in range(N)])

In [58]:
result_2

38.2341843822162

In [59]:
abs(result_2 - v) < eps

False

# 5) Формула Гаусса

In [215]:
x = sp.symbols('x')
n = 16
f = x*x*x*sp.exp(-0.01*n*x**(3/2))
a = 0.1*(n+3)
b = 0.25*(n+3)
eps = 0.001
N = 4

In [216]:
# Ортогональные многочлены Лежандра для поиска x
t = np.linspace(-1, 1, N)
P = (x*x - 1)**N
for _ in range(N):
    P = sp.diff(P)
P /= 2**n * np.prod([i for i in range(1, N+1)])

In [217]:
t_new = sp.solve(P, x)

In [218]:
x_num = [float(i*(b-a)/2 + (b+a)/2) for i in t_new]

In [219]:
x_num

[2.84052701289158, 3.8094729871084203, 2.0978807559784753, 4.552119244021525]

In [220]:
w = 1
for i in range(N):
    w *= (x - x_num[i])
w

(x - 4.55211924402153)*(x - 3.80947298710842)*(x - 2.84052701289158)*(x - 2.09788075597848)

In [221]:
dw = sp.diff(w, x)
C_list = [(w / (x - x_num[i]) / dw.subs({x:x_num[i]})) for i in range(N)]
C_list

[0.811929150996207*(x - 4.55211924402153)*(x - 3.80947298710842)*(x - 2.09788075597848),
 -0.811929150996207*(x - 4.55211924402153)*(x - 2.84052701289158)*(x - 2.09788075597848),
 -0.320553803568772*(x - 4.55211924402153)*(x - 3.80947298710842)*(x - 2.84052701289158),
 0.320553803568772*(x - 3.80947298710842)*(x - 2.84052701289158)*(x - 2.09788075597848)]

In [222]:
C_list_integrate = [sp.lambdify(x, i) for i in C_list]
C_list_integrate = [integrate.quad(i, a, b)[0] for i in C_list_integrate]

In [223]:
C_list_integrate

[0.9293068456791135,
 0.9293068456791447,
 0.49569315432087374,
 0.49569315432087063]

In [224]:
result = sum([C_list_integrate[i] * f.subs({x:x_num[i]}) for i in range(N)])

In [225]:
result

38.2359131591975

In [226]:
f_lam = sp.lambdify(x, f)
v, err = integrate.quad(f_lam, a, b)
print(v)

38.23592512595571


In [227]:
abs(result - v) < eps

True

# 6) Первый интеграл с помощью сплайнов

In [17]:
x = sp.symbols('x')
n = 16
f = x*x*x*sp.exp(-0.01*n*x**(3/2))
a = 0.1*(n+3)
b = 0.25*(n+3)
N = 8
a, b

(1.9000000000000001, 4.75)

In [4]:
h = (b - a) / N
x_num = np.linspace(a, b, N + 1)
x_num

array([1.9    , 2.25625, 2.6125 , 2.96875, 3.325  , 3.68125, 4.0375 ,
       4.39375, 4.75   ])

In [289]:
df = sp.diff(f, x)
M = sp.diff(df, x)

In [290]:
# Частный случай для равномерного распределения
result = (5*h*(f.subs({x:a}) + f.subs({x:b})) / 12 + 
          13*h*(f.subs({x:x_num[1]}) + f.subs({x:x_num[N - 1]})) / 12 +
          h*sum([f.subs({x:x_num[i]}) for i in range(2, N - 1)]) - 
          (h**3)*(2*M.subs({x:x_num[0]}) + M.subs({x:x_num[1]}) + M.subs({x:x_num[N-1]}) + 2 * M.subs({x:x_num[N]})) / 72)
result

38.2358631435758

In [291]:
# Общий случай
result = (1/2 * sum([h * (f.subs({x:x_num[i]}) + f.subs({x:x_num[i + 1]})) for i in range(N)]) - 
         1/24 * sum([(h ** 3) * (M.subs({x:x_num[i]}) + M.subs({x:x_num[i + 1]})) for i in range(N)]))
result

38.2355578945476

In [18]:
f_lam = sp.lambdify(x, f)
v, err = integrate.quad(f_lam, a, b)
print(v)

38.23592512595571


# 7) Метод Рунге–Ромберга

In [16]:
x = sp.symbols('x')
n = 16
f = x*x*x*sp.exp(-0.01*n*x**(3/2))
a = 0.1*(n+3)
b = 0.25*(n+3)
eps = 10 ** -8
N = 4
m = 2

In [17]:
def trapezoid_for_RR(f, N):
    x_num = np.linspace(a, b, N)
    
    return sum([(x_num[i+1] - x_num[i]) * (f.subs({x:x_num[i]}) + f.subs({x:x_num[i+1]})) / 2 for i in range(len(x_num) - 1)])

In [19]:
Y = trapezoid_for_RR(f, N)
Y_new = 0
while (abs(Y - Y_new) / (2 ** m - 1)) > eps:
    N *= 2
    Y = Y_new
    Y_new = trapezoid_for_RR(f, N)
    print(Y, Y_new, abs(Y - Y_new), N)
Y_new

0 38.2256583001972 38.2256583001972 16
38.2256583001972 38.2335222286320 0.00786392843483696 32
38.2335222286320 38.2353433705150 0.00182114188294236 64
38.2353433705150 38.2357819713947 0.000438600879746787 128
38.2357819713947 38.2358896176393 0.000107646244579485 256
38.2358896176393 38.2359162835976 2.66659582877082e-5 512
38.2359162835976 38.2359229196865 6.63608894058143e-6 1024
38.2359229196865 38.2359245749272 1.65524065209866e-6 2048
38.2359245749272 38.2359249882659 4.13338668181495e-7 4096
38.2359249882659 38.2359250915417 1.03275858975849e-7 8192
38.2359250915417 38.2359251173533 2.58115875340081e-8 16384


38.2359251173533

In [20]:
f_lam = sp.lambdify(x, f)
v, err = integrate.quad(f_lam, a, b)
print(v, err)

38.23592512595571 4.2450404442679567e-13


In [21]:
abs(Y_new - v) < eps

True

# 8) Несобственный интеграл

In [44]:
x = sp.symbols('x')
n = 16
f = (sp.sin(x + 0.1*n)) / ((x + 0.1*n)**2 * (x - n)**2) ** (1/3)
a = -0.1*n
b = 0.1*n
eps = 0.001
N = 4
m = 2
f

0.731004434553217*((0.625*x + 1)**2*(x - 16)**2)**(-0.333333333333333)*sin(x + 1.6)

In [45]:
alpha = 1
f_lam = sp.lambdify(x, f)
v = integrate.quad(f_lam, a, a + alpha)
while v[0] > (eps / 2):
    v = integrate.quad(f_lam, a, a + alpha)
    alpha /= 2
    print(v[0], alpha)

0.10594869263914483 0.5
0.04373508052813402 0.25
0.017479254764726084 0.125
0.006939449233585245 0.0625
0.0027523354510323905 0.03125
0.0010917382612653246 0.015625
0.00043313111284407085 0.0078125


In [46]:
def trapezoid_for_RR(f, a, b, N):
    x_num = np.linspace(a, b, N)
    
    return sum([(x_num[i+1] - x_num[i]) * (f.subs({x:x_num[i]}) + f.subs({x:x_num[i+1]})) / 2 for i in range(len(x_num) - 1)])

In [47]:
Y = trapezoid_for_RR(f, (a + alpha), b, N)
Y_new = 0
while (abs(Y - Y_new) / (2 ** m - 1)) > eps:
    N *= 2
    Y = Y_new
    Y_new = trapezoid_for_RR(f, (a + alpha), b, N)
    print(Y, Y_new, abs(Y - Y_new), N)
Y_new

0 0.265870184866147 0.265870184866147 8
0.265870184866147 0.272731686983688 0.00686150211754116 16
0.272731686983688 0.274696822797079 0.00196513581339081 32


0.274696822797079

In [48]:
v = integrate.quad(f_lam, a, b)
v[0]

0.2756857984201168

In [49]:
abs(Y_new - v[0]) < eps

True